## NEON data

Data downloaded from https://data.neonscience.org/ using api at https://data.neonscience.org/data-api/. More useful information at: https://data.neonscience.org/data-products/DP1.00094.001

Python/API: https://www.neonscience.org/resources/learning-hub/tutorials/neon-api-intro-requests-py

Downloading: https://www.neonscience.org/resources/learning-hub/tutorials/download-explore-neon-data

Cleaning: https://www.neonscience.org/resources/learning-hub/tutorials/clean-neon-ais-data

Cite provisional data as:
NEON (National Ecological Observatory Network). Soil water content and water salinity (DP1.00094.001), provisional data. Dataset accessed from https://data.neonscience.org/data-products/DP1.00094.001 on December 18, 2024. Data archived at [your DOI].

Cite Released data as:
NEON (National Ecological Observatory Network). Soil water content and water salinity (DP1.00094.001), RELEASE-2024. https://doi.org/10.48443/a8vy-y813. Dataset accessed from https://data.neonscience.org/data-products/DP1.00094.001/RELEASE-2024 on December 18, 2024.

In [2]:
import os
import neonutilities as nu
import rasterio
import glob
import requests
import json
import itertools


import datetime
from datetime import timedelta
import pandas as pd
import numpy as np
import h5py
from zeep import Client, Settings
from pprint import pprint
import zeep
import csv
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.dates import DateFormatter
from scipy.interpolate import RegularGridInterpolator
import json as js
import re
import setParams as p
from scipy.spatial import ConvexHull

In [3]:
#Every request begins with the server's URL
SERVER = 'http://data.neonscience.org/api/v0/'

In [4]:
#Site Code for Lower Teakettle
SITECODE = 'TEAK'
#Define the url, using the sites/ endpoint
url = SERVER+'sites/'+SITECODE
print(url)

http://data.neonscience.org/api/v0/sites/TEAK


In [5]:
#Request the url
site_request = requests.get(url)

#Convert the request to Python JSON object
site_json = site_request.json()

In [6]:
#Use the 'keys' method to view the component of the uppermost json dictionary
site_json.keys()


dict_keys(['data'])

In [7]:
#Access the 'data' component, and use the 'keys' method to view to componenets of the json data dictionary
site_json['data'].keys()

dict_keys(['siteCode', 'siteName', 'siteDescription', 'siteType', 'siteLatitude', 'siteLongitude', 'stateCode', 'stateName', 'domainCode', 'domainName', 'deimsId', 'releases', 'dataProducts'])

In [8]:
dict(itertools.islice(site_json['data'].items(),12))

{'siteCode': 'TEAK',
 'siteName': 'Lower Teakettle NEON',
 'siteDescription': 'Lower Teakettle NEON',
 'siteType': 'GRADIENT',
 'siteLatitude': 37.00583,
 'siteLongitude': -119.00602,
 'stateCode': 'CA',
 'stateName': 'California',
 'domainCode': 'D17',
 'domainName': 'Pacific Southwest',
 'deimsId': 'https://deims.org/16d02952-7cb5-487b-a5a3-7bc9ba0d557e',
 'releases': [{'release': 'RELEASE-2021',
   'generationDate': '2021-01-23T02:30:02Z',
   'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2021'},
  {'release': 'RELEASE-2022',
   'generationDate': '2022-01-20T17:39:46Z',
   'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2022'},
  {'release': 'RELEASE-2023',
   'generationDate': '2023-01-27T12:07:53Z',
   'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2023'},
  {'release': 'RELEASE-2024',
   'generationDate': '2024-01-27T00:04:25Z',
   'url': 'https://data.neonscience.org/api/v0/releases/RELEASE-2024'}]}

In [9]:
#View a data product dictionary
site_json['data']['dataProducts'][-3]

{'dataProductCode': 'DP3.30026.001',
 'dataProductTitle': 'Vegetation indices - spectrometer - mosaic',
 'availableMonths': ['2013-06', '2017-06', '2018-06', '2019-06', '2021-07'],
 'availableDataUrls': ['https://data.neonscience.org/api/v0/data/DP3.30026.001/TEAK/2013-06',
  'https://data.neonscience.org/api/v0/data/DP3.30026.001/TEAK/2017-06',
  'https://data.neonscience.org/api/v0/data/DP3.30026.001/TEAK/2018-06',
  'https://data.neonscience.org/api/v0/data/DP3.30026.001/TEAK/2019-06',
  'https://data.neonscience.org/api/v0/data/DP3.30026.001/TEAK/2021-07'],
 'availableReleases': [{'release': 'RELEASE-2024',
   'availableMonths': ['2013-06',
    '2017-06',
    '2018-06',
    '2019-06',
    '2021-07']}]}

In [11]:
#View product code and name for every available data product
for product in site_json['data']['dataProducts']:
    print(product['dataProductCode'],product['dataProductTitle'])

DP1.00001.001 2D wind speed and direction
DP1.00002.001 Single aspirated air temperature
DP1.00003.001 Triple aspirated air temperature
DP1.00004.001 Barometric pressure
DP1.00005.001 IR biological temperature
DP1.00006.001 Precipitation
DP1.00014.001 Shortwave radiation (direct and diffuse pyranometer)
DP1.00023.001 Shortwave and longwave radiation (net radiometer)
DP1.00024.001 Photosynthetically active radiation (PAR)
DP1.00033.001 Phenology images
DP1.00040.001 Soil heat flux plate
DP1.00041.001 Soil temperature
DP1.00042.001 Snow depth and understory phenology images
DP1.00066.001 Photosynthetically active radiation (quantum line)
DP1.00094.001 Soil water content and water salinity
DP1.00095.001 Soil CO2 concentration
DP1.00096.001 Soil physical and chemical properties, Megapit
DP1.00098.001 Relative humidity
DP1.10003.001 Breeding landbird point counts
DP1.10010.001 Coarse downed wood log survey
DP1.10017.001 Digital hemispheric photos of plot vegetation
DP1.10020.001 Ground beet

In [10]:
#Set the Ecosystem structure (CHM) data product
PRODUCTCODE = 'DP3.30015.001'
#Get available months of Ecosystem structure data products for TEAK site
#Loop through the 'dataProducts' list items (each one is a dictionary) at the site
for product in site_json['data']['dataProducts']: 
    #if a list item's 'dataProductCode' dict element equals the product code string
    if(product['dataProductCode'] == PRODUCTCODE): 
        #print the available months
        print('Available Months: ',product['availableMonths'])
        print('URLs for each Month:')
        #print the available URLs
        for url in product['availableDataUrls']:
            print(url)

Available Months:  ['2013-06', '2017-06', '2018-06', '2019-06', '2021-07', '2023-07']
URLs for each Month:
https://data.neonscience.org/api/v0/data/DP3.30015.001/TEAK/2013-06
https://data.neonscience.org/api/v0/data/DP3.30015.001/TEAK/2017-06
https://data.neonscience.org/api/v0/data/DP3.30015.001/TEAK/2018-06
https://data.neonscience.org/api/v0/data/DP3.30015.001/TEAK/2019-06
https://data.neonscience.org/api/v0/data/DP3.30015.001/TEAK/2021-07
https://data.neonscience.org/api/v0/data/DP3.30015.001/TEAK/2023-07


In [16]:
#Set the soil water content and water salinity
PRODUCTCODE = 'DP1.00094.001'
#Get available months of data products for TEAK site
#Loop through the 'dataProducts' list items (each one is a dictionary) at the site
for product in site_json['data']['dataProducts']: 
    #if a list item's 'dataProductCode' dict element equals the product code string
    if(product['dataProductCode'] == PRODUCTCODE): 
        #print the available months
        print('Available Months: ',product['availableMonths'])
        print('URLs for each Month:')
        #print the available URLs
        for url in product['availableDataUrls']:
            print(url)

Available Months:  ['2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07', '2024-08', '2024-09', '2024-10', '2024-11']
URLs for each Month:
https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-06
https://data.neonscience.org/api/

In [17]:
#Make request
product_request = requests.get(SERVER+'products/'+PRODUCTCODE)
product_json = product_request.json()
#Print keys for product data dictionary
print(product_json['data'].keys())
#Print code, name, and abstract of data product
print(product_json['data']['productCode'])
print(product_json['data']['productName'],'\n')
print(product_json['data']['productAbstract'])


dict_keys(['productCodeLong', 'productCode', 'productCodePresentation', 'productName', 'productDescription', 'productStatus', 'productCategory', 'productHasExpanded', 'productScienceTeamAbbr', 'productScienceTeam', 'productPublicationFormatType', 'productAbstract', 'productDesignDescription', 'productStudyDescription', 'productBasicDescription', 'productExpandedDescription', 'productSensor', 'productRemarks', 'themes', 'changeLogs', 'specs', 'keywords', 'biorepositoryCollections', 'releases', 'siteCodes'])
DP1.00094.001
Soil water content and water salinity 

Soil volumetric water content and an index of soil water ion content (salinity) are measured at various depths below the soil surface from approximately 6 cm down to 200 cm at non-permafrost sites (down to 300 cm at Alaskan sites). Soil moisture is an important component of the hydrologic cycle and is the dominant source of water for most plants and soil organisms making it a key indicator of drought. In addition, soil moisture st

In [13]:
#View keys of one site dictionary
print(product_json['data']['siteCodes'][0].keys())


dict_keys(['siteCode', 'availableMonths', 'availableDataUrls', 'availableReleases'])


In [18]:
#View available months and corresponding API urls, then save desired URL
for site in product_json['data']['siteCodes']:
    if(site['siteCode'] == SITECODE):
        for month in zip(site['availableMonths'],site['availableDataUrls']): #Loop through the list of months and URLs
            print(month[0],month[1]) 
            if(month[0] == '2024-11'): #If data is available for the desired month, save the URL
                data_url = month[1]

2018-06 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-06
2018-07 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-07
2018-08 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-08
2018-09 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-09
2018-10 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-10
2018-11 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-11
2018-12 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2018-12
2019-01 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2019-01
2019-02 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2019-02
2019-03 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2019-03
2019-04 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2019-04
2019-05 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2019-05
2019-06 https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2019-06
2019-07 http

In [19]:
print(data_url)

https://data.neonscience.org/api/v0/data/DP1.00094.001/TEAK/2024-11


In [20]:
#Make Request
data_request = requests.get(SERVER+'data/'+PRODUCTCODE+'/'+SITECODE+'/'+'2024-11')
data_json = data_request.json()

In [21]:
#Print dict key for 'data' element of data JSON
print(data_json['data'].keys())

dict_keys(['productCode', 'siteCode', 'month', 'release', 'packages', 'files'])


In [ ]:
#Make request with saved url
data_request = requests.get(data_url)
data_json = data_request.json()

In [22]:
#Print dict key for 'data' element of data JSON
print(data_json['data'].keys())

dict_keys(['productCode', 'siteCode', 'month', 'release', 'packages', 'files'])


In [23]:
#View keys and values in first file dict
for key in data_json['data']['files'][0].keys(): #Loop through keys of the data file dict
    print(key,':\t', data_json['data']['files'][0][key])


name :	 NEON.D17.TEAK.DP1.00094.001.002.503.030.SWS_30_minute.2024-11.expanded.20241203T231635Z.csv
size :	 587830
md5 :	 d6641a135fb6f9867b1459ffa09a024b
crc32 :	 None
crc32c :	 None
url :	 https://storage.googleapis.com/neon-publication/NEON.DOM.SITE.DP1.00094.001/TEAK/20241101T000000--20241201T000000/expanded/NEON.D17.TEAK.DP1.00094.001.002.503.030.SWS_30_minute.2024-11.expanded.20241203T231635Z.csv


In [24]:
#Display the names of the first 10 files
for file in data_json['data']['files'][:10]:
    print(file['name'])

NEON.D17.TEAK.DP1.00094.001.002.503.030.SWS_30_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.005.501.001.SWS_1_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.001.503.030.SWS_30_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.004.504.030.SWS_30_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.003.504.030.SWS_30_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.002.501.001.SWS_1_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.003.503.030.SWS_30_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.002.508.001.SWS_1_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.001.504.001.SWS_1_minute.2024-11.expanded.20241203T231635Z.csv
NEON.D17.TEAK.DP1.00094.001.001.503.001.SWS_1_minute.2024-11.expanded.20241203T231635Z.csv
